In [70]:
#Importando os modulos necessários
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
import numpy as np
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import HistGradientBoostingClassifier, HistGradientBoostingRegressor


#https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10510409/

#Principais variáveis para hipertensão: GENDER, AGERANGE, RACE, BMXBMI, KIDNEDISEASE, SMOKE, DIABETES, HPERTENSION, POVERT RATIO... EXPLORAR OUTROS

#https://www.sciencedirect.com/science/article/pii/S1279770723012459
#Principais variáveis para INFARTO: MARITAL STATUS, GENDER, AGE, TOTAL CHOLESTEROL, 
#BLOOD LEAD, BMI, SSTOLIC BLOOD PRESSURE, DIABETES, HDL, MEDICAL INSURANCE, HOUSEHOLD INCOME, SLEEP STATUS, POVERT RATIO... EXPLORAR OUTROS


#https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8160335/#:~:text=We%20used%20the%20same%20five,total%20cholesterol%20levels%2C%20as%20features.
#Principais variáveis para DIABETES: FAMILY HISTORY OF DIABETES, BMI, RACE, HYPERTENSION, TOTAL CHOLESTEROL... EXPLORAR OUTROS

#tirando o limite para facilitar visualização do df
pd.set_option('display.max_columns', None)


#ler df
df_demographic = pd.read_sas('P_DEMO.XPT', format='xport')


In [71]:
#Renomando as colunas para facilitar visualização
df_demographic = df_demographic.rename(columns ={'RIAGENDR': 'GENDER','RIDAGEYR': 'AGE', 'RIDRETH3': 'RACE', 
                                                 'DMDEDUC2': 'EDUCATION',  'RIDEXPRG': 'PRAGNANCY_STATUS',
                                                 'DMDMARTZ': 'MARITAL_STATUS', 
                                                 'INDFMPIR': 'POVERT_RATIO' })

In [72]:
#Transformando os valores de refused / dont know para NaN para facilitar o tratamento dos dados 
df_demographic['EDUCATION'] = df_demographic['EDUCATION'].replace([7, 9], np.nan)


#Transformando os valores de refused / dont know para NaN para facilitar o tratamento dos dados 
df_demographic['MARITAL_STATUS'] = df_demographic['MARITAL_STATUS'].replace([77, 99], np.nan)



In [73]:
#EXCLUINDO GESTANTES
df_demographic = df_demographic.loc[~df_demographic['PRAGNANCY_STATUS'].isin([1, 3])]


In [74]:
def impute_missing_values(df, target_col, classifier=True):
    """Imputa valores missing usando HistGradientBoostingClassifier/Regressor.
    
    Args:
        df (pd.DataFrame): DataFrame completo com todas as colunas.
        target_col (str): Nome da coluna alvo a ser imputada.
        classifier (bool): Se True, usa HistGradientBoostingClassifier, caso contrário usa HistGradientBoostingRegressor.
        
    Returns:
        pd.Series: Coluna com os valores missing imputados.
    """
    # Separando as linhas que possuem e não possuem o valor alvo (target_col)
    df_train = df[df[target_col].notna()]
    df_missing = df[df[target_col].isna()]

    # Se não houver valores missing, não há necessidade de imputar
    if df_missing.empty:
        return df[target_col]

    # Selecionando as features (todas as colunas exceto a target_col)
    X_train = df_train.drop(columns=[target_col])
    y_train = df_train[target_col]
    
    # Modelo HistGradientBoosting (classifier ou regressor)
    model = HistGradientBoostingClassifier() if classifier else HistGradientBoostingRegressor()
    
    # Treinando o modelo
    model.fit(X_train, y_train)
    
    # Prevendo os valores missing
    df_missing_pred = model.predict(df_missing.drop(columns=[target_col]))
    
    # Usando .loc para evitar o erro de cópia de fatia
    df.loc[df[target_col].isna(), target_col] = df_missing_pred
    
    return df[target_col]

# Imputando 'EDUCATION' (classificador)
df_demographic['EDUCATION'] = impute_missing_values(df_demographic, 'EDUCATION', classifier=True)

# Imputando 'MARITAL_STATUS' (classificador)
df_demographic['MARITAL_STATUS'] = impute_missing_values(df_demographic, 'MARITAL_STATUS', classifier=True)

# Imputando 'POVERT_RATIO' (regressor, assumindo que seja uma variável contínua)
df_demographic['POVERT_RATIO'] = impute_missing_values(df_demographic, 'POVERT_RATIO', classifier=False)

# Arredondando e convertendo as colunas categóricas
df_demographic['EDUCATION'] = df_demographic['EDUCATION'].round().astype('category')
df_demographic['MARITAL_STATUS'] = df_demographic['MARITAL_STATUS'].round().astype('category')

In [75]:
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score

def evaluate_imputation(df, target_col, classifier=True, test_size=0.2):
    """Avalia a precisão da imputação ao simular valores missing e comparar com os valores reais.
    
    Args:
        df (pd.DataFrame): DataFrame completo com todas as colunas.
        target_col (str): Nome da coluna alvo a ser avaliada.
        classifier (bool): Se True, avalia um classificador, caso contrário avalia um regressor.
        test_size (float): Proporção de valores a serem removidos para teste.
        
    Returns:
        float: Acurácia para classificação ou R^2 para regressão.
    """
    # Separando as linhas sem valores missing na target_col
    df_known = df[df[target_col].notna()].copy()

    # Determinando o número de amostras para remover
    n_samples = int(len(df_known) * test_size)
    
    # Removendo uma fração dos dados para simulação
    df_known.loc[df_known.sample(n_samples).index, target_col] = np.nan
    
    # Imputando os valores missing simulados
    imputed_col = impute_missing_values(df_known, target_col, classifier=classifier)
    
    # Extraindo os valores reais e imputados para comparação
    y_true = df[target_col].loc[df_known.index]
    y_pred = imputed_col.loc[df_known.index]

    if classifier:
        # Acurácia para classificação
        return accuracy_score(y_true, y_pred)
    else:
        # MSE e R^2 para regressão
        mse = mean_squared_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)
        return r2, mse

# Avaliando 'EDUCATION' (classificador)
education_accuracy = evaluate_imputation(df_demographic, 'EDUCATION', classifier=True)
print(f'Acurácia na imputação de EDUCATION: {education_accuracy:.4f}')

# Avaliando 'MARITAL_STATUS' (classificador)
marital_status_accuracy = evaluate_imputation(df_demographic, 'MARITAL_STATUS', classifier=True)
print(f'Acurácia na imputação de MARITAL_STATUS: {marital_status_accuracy:.4f}')

# Avaliando 'POVERT_RATIO' (regressor)
poverty_r2, poverty_mse = evaluate_imputation(df_demographic, 'POVERT_RATIO', classifier=False)
print(f'R² na imputação de POVERT_RATIO: {poverty_r2:.4f}, MSE: {poverty_mse:.4f}')

Acurácia na imputação de EDUCATION: 0.9293
Acurácia na imputação de MARITAL_STATUS: 0.9586
R² na imputação de POVERT_RATIO: 0.8988, MSE: 0.2481


In [76]:
#Excluir variáveis pouco importantes para a análise desejada (Ex. idioma, peso da amostra e similares)
df_demographic = df_demographic.drop(['SDDSRVYR', 'RIDSTATR', 'RIDAGEMN', 'MIAINTRP',
                                     'RIDRETH1','RIDEXMON', 'DMDBORN4', 'DMDYRUSZ', 'SIALANG', 
                                     'SIAPROXY','SIAINTRP',  'FIALANG','FIAPROXY', 'FIAINTRP', 'MIALANG', 
                                     'MIAPROXY', 'AIALANGA', 'WTINTPRP', 'SDMVPSU', 'SDMVSTRA', 'WTMECPRP', 'PRAGNANCY_STATUS'], axis=1 )


In [77]:
df_demographic['AGE'] = df_demographic['AGE'].astype('int')


In [78]:
df_demographic

,SEQN,GENDER,AGE,RACE,EDUCATION,MARITAL_STATUS,POVERT_RATIO
0,109263.0,1.0,2,6.0,4.0,3.0,4.660000
1,109264.0,2.0,13,1.0,2.0,3.0,0.830000
2,109265.0,1.0,2,3.0,2.0,3.0,3.060000
3,109266.0,2.0,29,6.0,5.0,3.0,5.000000
5,109268.0,2.0,18,3.0,4.0,3.0,1.660000
...,...,...,...,...,...,...,...
15555,124818.0,1.0,40,4.0,5.0,1.0,3.820000
15556,124819.0,1.0,2,4.0,4.0,3.0,0.070000
15557,124820.0,2.0,7,3.0,4.0,3.0,1.220000
15558,124821.0,1.0,63,4.0,2.0,2.0,3.710000


In [79]:
df_demographic.to_csv('dados_demograficos_tratados.csv', index=False)
